In [1]:
import cifar10
import matplotlib.pyplot as plt

In [2]:
cifar10.data_patch='data/Cifar-10/'

In [3]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [6]:
print("images_train",images_train.shape)
print("cls_train",cls_train.shape)
print("labels_train",labels_train.shape)

images_train (50000, 32, 32, 3)
cls_train (50000,)
labels_train (50000, 10)


In [7]:
print("images_test",images_test.shape)
print("cls_test",cls_test.shape)
print("labels_test",labels_test.shape)

images_test (10000, 32, 32, 3)
cls_test (10000,)
labels_test (10000, 10)


In [8]:
train_pixel=images_train.reshape(50000,3072)
test_pixel=images_test.reshape(10000,3072)

In [9]:
from sklearn.decomposition import PCA

In [10]:
pca=PCA()
pca.fit_transform(train_pixel)

array([[-6.40101763e+00,  2.72903876e+00,  1.50171052e+00, ...,
        -7.62287826e-04, -1.16359323e-03, -3.49768400e-04],
       [ 8.29783385e-01, -9.49943219e-01,  6.00375260e+00, ...,
         1.08434589e-04,  5.76715387e-04, -7.71005927e-04],
       [ 7.73019978e+00, -1.15221023e+01, -2.75362051e+00, ...,
        -1.04460109e-03, -2.14105740e-04, -3.93469526e-04],
       ...,
       [ 7.07346587e-01, -1.13425212e+01, -3.93914878e-01, ...,
        -3.80341568e-04,  1.86432348e-03,  1.44785229e-03],
       [ 1.13679090e+01, -3.37597765e+00,  5.56619766e+00, ...,
        -4.67096832e-04,  7.44972963e-04, -1.67065056e-03],
       [ 4.19279645e+00, -1.18418848e+00, -4.44596297e+00, ...,
         9.67968258e-04,  2.40363971e-04, -2.80506224e-05]])

In [11]:
k=0
total=sum(pca.explained_variance_)
current=0
while current/total <0.99:
    current+=pca.explained_variance_[k]
    k=k+1
k

658

In [12]:
pca = PCA(n_components=k,whiten=True)
transformed_data=pca.fit_transform(train_pixel)
transformed_data.shape

(50000, 658)

In [13]:
transformed_test=pca.transform(test_pixel)

In [14]:
X_train,X_test,Y_train,Y_test=transformed_data,transformed_test,labels_train,labels_test

In [15]:
Y_train[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [16]:
labels_train[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [17]:
Y_test[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [18]:
from keras.layers import Dense
from keras.models import Sequential

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
model=Sequential()

In [20]:
layer1=Dense(units=750,activation='relu',input_dim=k)
model.add(layer1)
layer2=Dense(units=500,activation='relu')
model.add(layer2)
layer3=Dense(units=250,activation='relu')
model.add(layer3)
output_layer=Dense(units=10,activation='sigmoid')
model.add(output_layer)

In [21]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(X_train,Y_train,epochs=19,batch_size=50,validation_data=(X_test,Y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/19
50000/50000 [==============================] - 15s 296us/step - loss: 1.8209 - acc: 0.3462 - val_loss: 1.5988 - val_acc: 0.4339
Epoch 2/19
50000/50000 [==============================] - 13s 251us/step - loss: 1.3258 - acc: 0.5247 - val_loss: 1.5046 - val_acc: 0.4729
Epoch 3/19
50000/50000 [==============================] - 14s 289us/step - loss: 0.9588 - acc: 0.6516 - val_loss: 1.5758 - val_acc: 0.4836
Epoch 4/19
50000/50000 [==============================] - 14s 286us/step - loss: 0.6475 - acc: 0.7631 - val_loss: 1.8581 - val_acc: 0.4841
Epoch 5/19
50000/50000 [==============================] - 15s 303us/step - loss: 0.4373 - acc: 0.8415 - val_loss: 2.1738 - val_acc: 0.4849
Epoch 6/19
50000/50000 [==============================] - 13s 262us/step - loss: 0.3208 - acc: 0.8849 - val_loss: 2.2981 - val_acc: 0.4895
Epoch 7/19
50000/50000 [==============================] - 14s 272us/step - loss: 0.2489 - acc: 0.9160 - val_loss: 2.

In [23]:
print('training_score',model.evaluate(X_train,Y_train))
print('testing_score',model.evaluate(X_test,Y_test))

50000/50000 [==============================] - 3s 60us/step
training_score [0.07293685972437262, 0.9783]
10000/10000 [==============================] - 1s 54us/step
testing_score [3.593381024169922, 0.48]
